In [14]:
import pandas as pd
import pymysql

connection = pymysql.connect(
    host='MacBook-Air-de-Pablo.local',
    user='root',
    password='MSa75047&',
    database='sakila'
)

query = """
SELECT
    f.film_id,
    f.title,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.rating,
    f.special_features,
    COUNT(r.rental_id) AS total_rentals,
    AVG(DATEDIFF(r.return_date, r.rental_date)) AS avg_rental_duration,
    AVG(f.rental_rate) AS avg_rental_rate,
    AVG(f.rental_rate * DATEDIFF(r.return_date, r.rental_date)) AS avg_rental_revenue
FROM
    film f
JOIN
    inventory i ON f.film_id = i.film_id
JOIN
    rental r ON i.inventory_id = r.inventory_id
GROUP BY
    f.film_id,
    f.title,
    f.rental_duration,
    f.rental_rate,
    f.length,
    f.rating,
    f.special_features
"""

df = pd.read_sql(query, connection)

df

/var/folders/5y/_5l3y2p900scjg2bd7r52_lh0000gn/T/ipykernel_43394/192815207.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


,film_id,title,rental_duration,rental_rate,length,rating,special_features,total_rentals,avg_rental_duration,avg_rental_rate,avg_rental_revenue
0,1,ACADEMY DINOSAUR,6,0.99,86,PG,"Deleted Scenes,Behind the Scenes",23,5.0909,0.99,5.040000
1,2,ACE GOLDFINGER,3,4.99,48,G,"Trailers,Deleted Scenes",7,5.6667,4.99,28.276667
2,3,ADAPTATION HOLES,7,2.99,50,NC-17,"Trailers,Deleted Scenes",12,3.4167,2.99,10.215833
3,4,AFFAIR PREJUDICE,5,2.99,117,G,"Commentaries,Behind the Scenes",23,4.7273,2.99,14.134545
4,5,AFRICAN EGG,6,2.99,130,G,Deleted Scenes,12,7.0909,2.99,21.201818
...,...,...,...,...,...,...,...,...,...,...,...
953,996,YOUNG LANGUAGE,6,0.99,183,G,"Trailers,Behind the Scenes",7,4.5714,0.99,4.525714
954,997,YOUTH KICK,4,0.99,179,NC-17,"Trailers,Behind the Scenes",6,5.6667,0.99,5.610000
955,998,ZHIVAGO CORE,6,0.99,105,NC-17,Deleted Scenes,9,5.8750,0.99,5.816250
956,999,ZOOLANDER FICTION,5,2.99,101,R,"Trailers,Deleted Scenes",17,5.5882,2.99,16.708824


In [17]:
query2 = """
SELECT
    f.film_id,
    f.title,
    IF(COUNT(r.rental_id) > 0, 1, 0) AS rented_last_month
FROM
    film f
LEFT JOIN
    inventory i ON f.film_id = i.film_id
LEFT JOIN
    rental r ON i.inventory_id = r.inventory_id
    AND YEAR(r.rental_date) = YEAR(CURRENT_DATE - INTERVAL 1 MONTH)
    AND MONTH(r.rental_date) = MONTH(CURRENT_DATE - INTERVAL 1 MONTH)
GROUP BY
    f.film_id,
    f.title
"""

target = pd.read_sql(query2, connection)

/var/folders/5y/_5l3y2p900scjg2bd7r52_lh0000gn/T/ipykernel_43394/3966082281.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  target = pd.read_sql(query2, connection)


In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df[['film_id', 'title']]
y = target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: Found input variables with inconsistent numbers of samples: [958, 1000]